---
title: C++ 简单测试框架
tags: 小书匠,c++,test,gtest,胡船长,define,macro,attribute,constructor
grammar_cjkRuby: true
renderNumberedHeading: true
---

[toc]

# C++ 简单测试框架

## 使用宏来定义函数

里面有许多 trick

1. args... 表示可变参数
2. ##args
3. a##b 表示拼接符号

In [1]:
%%file test.cpp

#include <iostream>
#define LOG(frm, args...) \
{ \
    printf("[%s : %d : %s] ", __FILE__, __LINE__, __func__); \
    printf(frm, ##args); \
    printf("\n"); \
} \
// ## 用于将两个符号拼接起来
#define Cate(a, b) a##b

int main()
{
    int a = 0;
    int abc = 123;
    LOG("hello ");
    LOG("hello %d", Cate(a, bc)); // 显示的是 123，因为 a 和 bc 拼接起来变成了 abc
    return 0;
}

Overwriting test.cpp


In [2]:
!g++ test.cpp -o test && ./test

[test.cpp : 16 : main] hello 
[test.cpp : 17 : main] hello 123


注意，代码中使用的 bc 是没有定义的，但是也不会报错。因为预处理阶段 bc 就被宏替换掉了。而预处理阶段不进行语法检查，编译阶段才检查。我么可以使用 `-E` 参数查看预处理后的待编译源码

In [3]:
!g++ test.cpp -E > test.i # 将其写入 test.i 文件中

In [4]:
!cat test.i | grep "main()" -A 10 -B 10 # 查看 main 函数的内容

extern __attribute__ ((__visibility__("default"))) wostream wcout;

extern __attribute__ ((__visibility__("default"))) ostream cerr;
extern __attribute__ ((__visibility__("default"))) wostream wcerr;
extern __attribute__ ((__visibility__("default"))) ostream clog;
extern __attribute__ ((__visibility__("default"))) wostream wclog;

} }
# 3 "test.cpp" 2
# 12 "test.cpp"
int main()
{
    int a = 0;
    int abc = 123;
    { printf("[%s : %d : %s] ", "test.cpp", 16, __func__); printf("hello "); printf("\n"); };
    { printf("[%s : %d : %s] ", "test.cpp", 17, __func__); printf("hello %d", abc); printf("\n"); };
    return 0;
}


可以看到，宏的位置已经被替换掉了。

## 完整实现

希望 `RUN_ALL_TESTS()` 函数运行后，所有的测试用例都会运行。因此我们可以用某个函数数组来保存所有的函数，然后 `RUN_ALL_TESTS` 的作用就是遍历这个函数数组。

### 封装颜色

In [5]:
#define GEN(msg) "\033[0;32;32m" msg "\033[m"
#define RED(msg) "\033[0;32;31m" msg "\033[m"

printf(GEN("hello"));
printf(RED(" world"))

hello world

19

In [6]:
#define EXPECT(a, cmp, b) {  \
    if(!(a cmp b)) { \
        printf(RED("EXPECT: " #a " " #cmp " " #b "\n")); \
    } \
}

#define EXPECT_EQ(a, b) EXPECT(a, ==, b)
#define EXPECT_LT(a, b) EXPECT(a, <,  b)
#define EXPECT_GT(a, b) EXPECT(a, >, b)
#define EXPECT_NE(a, b) EXPECT(a, !=, b)

EXPECT_EQ(1, 1);
EXPECT_EQ(1, 2);
EXPECT_GT(1, 2);
EXPECT_NE(1, 2);

EXPECT: 1 == 2
EXPECT: 1 > 2


In [2]:
struct {
    void (*func)();
    const char* name;
} func_array[1000];
int n_func = 0;

void RUN_ALL_TESTS()
{
    for(int i=0; i<n_func; i++)
    {
        printf(GEN("[ RUN %s ]"), func_array[i].name);
        func_array[i].func();
    }
}

input_line_8:8:1: error: function definition is not allowed here
{
^


Interpreter Error: 

那么问题变成了，那个函数填充了这个数组？在什么时候填充的？

我们希望 TEST 函数被调用的时候填充数组。

1. TEST 是一个宏。它会创建一个 test_xx 函数
2. TEST 需要把 test_xx 函数添加到 `func_array` 中，这个过程应该是在 main 函数之前完成的，因此需要用到 `__attribute__((constructor)`

In [10]:
#define TEST(func) \
void  \
__attribute__((constructor)) \
register_test_##func() \
{ \
    add_func(func, #func); \
} \
void test_##func()

void add_func(void (*pt)(), const char* name) {
    func_array[n_func] = {pt, name};
    n_func++;
}

input_line_15:10:5: error: use of undeclared identifier 'func_array'
    func_array[n_func] = {pt, name};
    ^
input_line_15:10:16: error: use of undeclared identifier 'n_func'
    func_array[n_func] = {pt, name};
               ^
input_line_15:11:5: error: use of undeclared identifier 'n_func'
    n_func++;
    ^


Interpreter Error: 

In [3]:
%%file test.h

#ifndef TEST_H
#define TEST_H
#include <iostream>

#define GEN(msg) "\033[0;32;32m" msg "\033[m"
#define RED(msg) "\033[0;32;31m" msg "\033[m"

#define EXPECT(a, cmp, b) {  \
    __typeof(a) _a = (a), _b = (b); \
    if(!((_a) cmp (_b))) { \
        printf(RED("EXPECT: %s %s %s. ACTUAL: %d %s %d\n"), #a, #cmp, #b, _a, #cmp, _b); \
    } \
} 

#define EXPECT_EQ(a, b) EXPECT(a, ==, b)
#define EXPECT_LT(a, b) EXPECT(a, <,  b)
#define EXPECT_GT(a, b) EXPECT(a, >, b)
#define EXPECT_NE(a, b) EXPECT(a, !=, b)

struct {
    void (*func)();
    const char* name;
} func_array[1000];
int n_func = 0;

void RUN_ALL_TESTS()
{
    for(int i=0; i<n_func; i++)
    {
        printf(GEN("[ RUN %s ]\n"), func_array[i].name);
        func_array[i].func();
    }
}

#define TEST(func) \
void test_##func(); \
void  \
__attribute__((constructor)) \
register_test_##func() \
{ \
    add_func(test_##func, #func); \
}; \
void test_##func()

void add_func(void (*func)(), const char* name) {
    func_array[n_func].func = func;
    func_array[n_func].name = name;
    n_func++;
}
#endif

Overwriting test.h


In [4]:
%%file test.cpp

#include <iostream>
#include "test.h"

int add(int a, int b) {
    return a + b;
}

TEST(add1) 
{
    EXPECT_EQ(add(0,1), 2);
    EXPECT_GT(add(0,1), 0);
    EXPECT_LT(add(0,1), 2);
    EXPECT_NE(add(0,1), 1);
}

TEST(add2) 
{
    EXPECT_EQ(add(0,1), 1);
    EXPECT_GT(add(0,1), 0);
    EXPECT_LT(add(0,1), 2);
    EXPECT_NE(add(0,1), 2);
}

int main()
{
    printf("run test!\n");
    RUN_ALL_TESTS();
    return 0;
}

Overwriting test.cpp


In [5]:
!g++ test.cpp test.h

clang: warning: treating 'c-header' input as 'c++-header' when in C++ mode, this behavior is deprecated [-Wdeprecated]


In [6]:
!./a.out

run test!
[ RUN add1 ]
EXPECT: add(0,1) == 2. ACTUAL: 1 == 2
EXPECT: add(0,1) != 1. ACTUAL: 1 != 1
[ RUN add2 ]


# References

- [C++船长免费课程 Google测试框架实现_zw1996的博客-CSDN博客_船长c++](https://blog.csdn.net/zw1996/article/details/109519628)